# Aula 10 - MultiDoc Q&A

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)


# Enunciado exercício


Implementar um pipeline multidoc QA: 

dado uma pergunta do usuário, buscamos em uma grande coleção as passagens mais relevantes e as enviamos para um sistema agregador, que irá gerar uma resposta final.
Avaliar no dataset do IIRC
Métrica principal: F1
Limitar dataset de teste para 50 exemplos para economizar.
Usar o gpt-3.5-turbo como modelo agregador. Usar vicuna-13B como alternativa open-source:
https://huggingface.co/helloollel/vicuna-13b 
https://chat.lmsys.org/ 
Dicas:
Se inspirar no pipeline do Visconde: https://github.com/neuralmind-ai/visconde 



Fonte: foi usado como apoio e base de "inspiração" o código relativo ao teste com o dataset IIRC em https://github.com/neuralmind-ai/visconde



# Organizando o ambiente

## Importações

In [2]:
import requests  # para Llama
import time # para Llama

In [3]:
import re

In [4]:
from tqdm import tqdm

In [5]:
import io

In [6]:
import spacy

In [7]:
os.environ['JVM_PATH'] = '/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [8]:
from pyserini.index import IndexReader

In [90]:
from pyserini.search.lucene import LuceneSearcher

In [9]:
import time


In [10]:
import os
import openai

from transformers import GPT2Tokenizer

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
import string

In [13]:
import gzip

In [14]:
import getpass

In [15]:
import gc

In [16]:
import json, time

In [17]:
from psutil import virtual_memory

In [18]:
import pandas as pd
import os

In [19]:
import random
import numpy as np


In [20]:
import pickle

In [21]:
from statistics import mean, stdev

In [98]:
import torch

In [99]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BatchEncoding

In [100]:
from torch.utils.data import DataLoader

In [101]:
from torch.utils.data import Dataset

## Definindo paths

In [22]:
DIRETORIO_LOCAL = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local'
DIRETORIO_TRABALHO = F'{DIRETORIO_LOCAL}/multidocqa'
DIRETORIO_IIRC = F'{DIRETORIO_LOCAL}/iirc'


In [23]:
DIRETORIO_CARDIN = f"{DIRETORIO_TRABALHO}/cardin"
CAMINHO_CARDIN = f"{DIRETORIO_CARDIN}/contents.jsonl"

In [92]:
DIRETORIO_INDICE = f"{DIRETORIO_TRABALHO}/indexes/iirc-index"

In [24]:
if os.path.exists(DIRETORIO_IIRC):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_IIRC)
    print('pasta criada!')


pasta já existia!


In [25]:
if os.path.exists(DIRETORIO_CARDIN):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_CARDIN)
    print('pasta criada!')


pasta já existia!


In [26]:
if os.path.exists(DIRETORIO_TRABALHO):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_TRABALHO)
    print('pasta criada!')


pasta já existia!


Outras inicializações

In [104]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Fixando as seeds

In [105]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  # torch.backends.cudnn.deterministic = True
  # torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [106]:
num_semente = 13 

In [107]:
inicializa_seed(num_semente)

## Preparando para debug e display

In [30]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [31]:
config_display()

# Baixando o dataset IIRC

In [ ]:
raise Exception("Pular para 'test_set = json.load' se dados já tiverem sido baixados!")

In [ ]:
if not os.path.exists(f"{DIRETORIO_IIRC}/iirc_test.json"):
    !wget wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json 
    !mv iirc_test.json {DIRETORIO_IIRC}
    print('Baixado')
else:
    print('Já existia arquivo!')

--2023-05-14 22:07:28--  http://wget/
Resolving wget (wget)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘wget’
--2023-05-14 22:07:28--  https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 52.218.229.241, 52.92.178.82, 52.92.165.162, ...
Connecting to iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)|52.218.229.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874825 (2,7M) [application/json]
Saving to: ‘iirc_test.json’

iirc_test.json      100%[===================>]   2,74M  2,06MB/s    in 1,3s    

2023-05-14 22:07:31 (2,06 MB/s) - ‘iirc_test.json’ saved [2874825/2874825]

FINISHED --2023-05-14 22:07:31--
Total wall clock time: 2,3s
Downloaded: 1 files, 2,7M in 1,3s (2,06 MB/s)
Baixado


In [ ]:
!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_train_dev.tgz 
!mv iirc_train_dev.tgz {DIRETORIO_IIRC}


--2023-05-14 22:03:31--  https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_train_dev.tgz
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 52.218.196.129, 52.218.220.89, 52.92.178.114, ...
Connecting to iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)|52.218.196.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5713428 (5,4M) [application/gzip]
Saving to: ‘iirc_train_dev.tgz’

iirc_train_dev.tgz  100%[===================>]   5,45M  3,03MB/s    in 1,8s    

2023-05-14 22:03:34 (3,03 MB/s) - ‘iirc_train_dev.tgz’ saved [5713428/5713428]



In [37]:
!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz 
!mv context_articles.tar.gz {DIRETORIO_IIRC}


--2023-05-14 22:03:49--  https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 52.92.139.50, 3.5.84.169, 3.5.82.198, ...
Connecting to iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)|52.92.139.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 385263479 (367M) [application/x-gzip]
Saving to: ‘context_articles.tar.gz’

context_articles.ta 100%[===================>] 367,42M  14,8MB/s    in 27s     

2023-05-14 22:04:17 (13,7 MB/s) - ‘context_articles.tar.gz’ saved [385263479/385263479]



In [43]:
!tar xvfz {DIRETORIO_IIRC}/context_articles.tar.gz -C  {DIRETORIO_IIRC}

context_articles.json


In [44]:
!tar xvfz {DIRETORIO_IIRC}/iirc_train_dev.tgz -C  {DIRETORIO_IIRC}

._iirc_train_dev
iirc_train_dev/
iirc_train_dev/._dev.json
iirc_train_dev/dev.json
iirc_train_dev/._README
iirc_train_dev/README
iirc_train_dev/._train.json
iirc_train_dev/train.json


In [32]:
test_set = json.load(open(f"{DIRETORIO_IIRC}/iirc_test.json",'r'))

In [33]:
context_articles = json.load(open(f"{DIRETORIO_IIRC}/context_articles.json",'r'))

In [34]:
list(context_articles.keys())[:4]

['san diego padres',
 'john bruce (historiographer)',
 'jack nicholson',
 'british raj']

In [35]:
context_articles['jack nicholson'][:1000]

'John Joseph Nicholson (born April 22, 1937) is an American actor and filmmaker. He is known for playing a wide range of starring or supporting roles, including comedy, romance, and darkly comic portrayals of anti-heroes and villainous characters. In many of his films, he played the "eternal <a href="Outlaw%20%28stock%20character%29">outsider</a>, the <a href="Sardonicism">sardonic</a> <a href="Vagrancy">drifter</a>", someone who rebels against the social structure.\n\nHis most known and celebrated films include the road drama <i><a href="Easy%20Rider">Easy Rider</a></i> (1969); the dramas <i><a href="Five%20Easy%20Pieces">Five Easy Pieces</a></i> (1970) and <i><a href="One%20Flew%20Over%20the%20Cuckoo%27s%20Nest%20%28film%29">One Flew Over the Cuckoo\'s Nest</a></i> (1975); the comedy-dramas <i><a href="Carnal%20Knowledge">Carnal Knowledge</a></i> (1971), <i><a href="The%20Last%20Detail">The Last Detail</a></i> (1973), <i><a href="Terms%20of%20Endearment">Terms of Endearment</a></i> (

In [36]:
# Removero html igual está na base IIRC é bem mais rápido que usar o BeuatifulSoup
# https://github.com/jferguson144/IIRC-baseline/blob/main/util.py
# dica do Carísio 
def remove_html(html):
  # Acrescentei replace '\xa0' 
  return re.sub("<[^>]*>", "", html).strip() #.replace('\xa0','')


In [37]:
remove_html('70\xa0million Prizzi\'s honor.')

"70\xa0million Prizzi's honor."

In [38]:
%%time
for titulo in context_articles.keys():
  context_articles[titulo] = remove_html(context_articles[titulo])


CPU times: user 3.97 s, sys: 0 ns, total: 3.97 s
Wall time: 3.97 s


In [39]:
context_articles['jack nicholson'][:1000], context_articles['jack nicholson'][-300:]

('John Joseph Nicholson (born April 22, 1937) is an American actor and filmmaker. He is known for playing a wide range of starring or supporting roles, including comedy, romance, and darkly comic portrayals of anti-heroes and villainous characters. In many of his films, he played the "eternal outsider, the sardonic drifter", someone who rebels against the social structure.\n\nHis most known and celebrated films include the road drama Easy Rider (1969); the dramas Five Easy Pieces (1970) and One Flew Over the Cuckoo\'s Nest (1975); the comedy-dramas Carnal Knowledge (1971), The Last Detail (1973), Terms of Endearment (1983), Prizzi\'s Honor (1985), As Good as It Gets (1997), About Schmidt (2002), and The Bucket List (2007); the neo-noir mystery Chinatown (1974); the horror film The Shining (1980); the biopic Reds (1981); the fantasy comedy The Witches of Eastwick (1987); the superhero film Batman (1989); the legal drama A Few Good Men (1992); the romantic horror film Wolf (1994); the sc

In [40]:
%%time
documents = []
all_titles = []

for item in test_set:
    if item['title'].lower() not in all_titles:
        documents.append({
                "title": item['title'],
                "content": item["text"]
            }
        )
        all_titles.append(item['title'].lower())
    for link in item["links"]:
        if link['target'].lower() in context_articles and link['target'].lower() not in all_titles:
            documents.append({
                "title": link['target'],
                "content": context_articles[link['target'].lower()]
            })
            all_titles.append(link['target'].lower())
        # else:
            # print(link['target'].lower())

CPU times: user 166 ms, sys: 0 ns, total: 166 ms
Wall time: 166 ms


In [41]:
documents_dict = {}
for doc in documents:
    documents_dict[doc['title']] = doc['content']

In [42]:
len(documents_dict)

7028

In [43]:
documents_dict['Palici']

"The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia plain, and as a result these twin brothers were associated with geysers and the underworld. There was also a shrine to the Palaci in Palacia, where people could subject themselves or others to tests of reliability through divine judgement; passing meant that an oath could be trusted. The mythological lineage of the Palici is uncertain; one legend made the Palici the sons of Zeus, or possibly Hephaestus, by Aetna or Thalia, but another claimed that the Palici were the sons of the Sicilian deity Adranus.\n"

In [44]:
len(documents), len(all_titles)

(7028, 7028)

In [45]:
documents[0], documents[2]['title'], documents[2]['content'][:1000]

({'title': 'Palici',
  'content': "The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia plain, and as a result these twin brothers were associated with geysers and the underworld. There was also a shrine to the Palaci in Palacia, where people could subject themselves or others to tests of reliability through divine judgement; passing meant that an oath could be trusted. The mythological lineage of the Palici is uncertain; one legend made the Palici the sons of Zeus, or possibly Hephaestus, by Aetna or Thalia, but another claimed that the Palici were the sons of the Sicilian deity Adranus.\n"},
 'Chthonic',
 'Chthonic (, ; from , "in, under, or beneath the earth", from "earth") literally means "subterranean", but th

In [46]:
all_titles[0], all_titles[2]

('palici', 'chthonic')

# Select questions

## Understanding dataset

In [119]:
# Convertendo o JSON para DataFrame
df = pd.DataFrame(test_set)

In [120]:
df.shape

(514, 4)

In [121]:
df.head()

,questions,text,links,title
0,"[{'answer': {'type': 'span', 'answer_spans': [...","The Palici (Παλικοί in Greek), or Palaci, were...","[{'target': 'Sicily', 'indices': [68, 76]}, {'...",Palici
1,"[{'answer': {'answer_value': '5', 'type': 'val...","Messe was born in Mesagne, in the Province of ...","[{'target': 'Mesagne', 'indices': [18, 25]}, {...",Giovanni Messe
2,"[{'answer': {'answer_value': '2', 'type': 'val...",Casino Royale also takes credit for the greate...,"[{'target': 'Ursula Andress', 'indices': [167,...",Casino Royale (1967 film)
3,"[{'answer': {'type': 'span', 'answer_spans': [...",Brunt returned to first-team action after eigh...,"[{'target': 'Anterior cruciate ligament', 'ind...",Chris Brunt
4,"[{'answer': {'type': 'span', 'answer_spans': [...",Wilhelm Müller was born on 7 October 1794 at D...,"[{'target': 'Dessau', 'indices': [45, 51]}, {'...",Wilhelm Müller


In [122]:

test_set[2]

{'questions': [{'answer': {'answer_value': '2',
    'type': 'value',
    'answer_unit': 'years'},
   'question': 'How long had Angela Scoular been acting professionally when she appeared in the movie "On Her Majesty\'s Secret Service"?',
   'context': [{'text': "Angela Scoular appeared as Ruby Bartlett in On Her Majesty's Secret Service",
     'passage': 'main',
     'indices': [418, 493]},
    {'text': "laying Ruby in 1969's On Her Majesty's Secret Service",
     'passage': 'Angela Scoular',
     'indices': [509, 562]},
    {'text': 'Scoular played Buttercup in the 1967',
     'passage': 'Angela Scoular',
     'indices': [366, 402]}],
   'question_links': ['Angela Scoular',
    "On Her Majesty's Secret Service (film)"]}],
 'text': "Casino Royale also takes credit for the greatest number of actors in a Bond film either to have appeared or to go on to appear in the rest of the Eon series – besides Ursula Andress in Dr. No, Vladek Sheybal appeared as Kronsteen in From Russia with Love, B

In [205]:
test_set[6]

{'questions': [{'answer': {'type': 'none'},
   'question': 'What albums were ranked higher than "It Takes a Nation of Millions to Hold Us Back" in Rolling Stone\'s the 500 Greatest Albums of All Time?',
   'context': [{'text': 'In 2003, Rolling Stone ranked the album number 48 on its list of the 500 Greatest Albums of All Time',
     'passage': 'main',
     'indices': [270, 370]}],
   'question_links': []}],
 'text': 'Mojo stated upon the album\'s 2000 European reissue, "Responsible for the angriest polemic since The Last Poets...[they] revolutionized the music, using up to 80 backing tracks in the sonic assault...to these ears PE sound like the greatest rock\'n\'roll band in history". In 2003, Rolling Stone ranked the album number 48 on its list of the 500 Greatest Albums of All Time, making it the highest-ranked of the 27 hip hop albums included on the list, maintaining the rating in a 2012 revised list. Time magazine hailed it as one of the 100 greatest albums of all time in 2006. K

In [123]:
test_set[24]

{'questions': [{'answer': {'answer_value': 'yes', 'type': 'binary'},
   'question': "Had the Chicago White Sox been around longer than the Birminham Barons when he was assigned to the White Sox's minor league system?",
   'context': [{'text': "Jordan then further surprised the sports world by signing a minor league baseball contract with the Chicago White Sox. He reported to spring training and was assigned to the team's minor league system on March 31, 1994",
     'passage': 'main',
     'indices': [0, 218]},
    {'text': 'He had an unspectacular professional baseball career for the Birmingham Barons, a Chicago White Sox farm team',
     'passage': 'main',
     'indices': [529, 638]},
    {'text': ' In 1900, with the approval of Western League president Ban Johnson, Charles Comiskey moved the Saints into his hometown neighborhood of Bridgeport, Chicago, where they became known as the White Stockings',
     'passage': 'Chicago White Sox',
     'indices': [2147, 2351]},
    {'text': 'Th

In [124]:
test_set[0]

{'questions': [{'answer': {'type': 'span',
    'answer_spans': [{'text': 'sky and thunder god',
      'passage': 'zeus',
      'type': 'answer',
      'start': 83,
      'end': 102}]},
   'question': 'What is Zeus know for in Greek mythology?',
   'context': [{'text': 'he Palici the sons of Zeus',
     'passage': 'main',
     'indices': [684, 710]},
    {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]},
    {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion',
     'passage': 'Zeus',
     'indices': [0, 110]}],
   'question_links': ['Greek mythology', 'Zeus']}],
 'text': "The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia 

## Selecting records

Por falta de tempo para testes mais avançados, e como se deseja passar pelo pipeline de forma completa e compreendida, selecionaremos respostas do tipo Sim/Não por serem aparentemente mais simples para a nossa compreensão (desenvolvedor)

In [66]:
questions_data = []

# Percorrendo cada JSON no test_set
for ndx_doc, json_data in enumerate(test_set):
    # Percorrendo cada questão no JSON
    for dict_question in json_data['questions']:
        question_data={}
        question_data['doc_pos'] = ndx_doc
        question_data['doc_title'] = json_data['title']
        question_data['doc_len_text'] = len(json_data['text'])
        question_data['doc_text'] = json_data['text']
        question_data['answer_type'] = dict_question['answer']['type']
        if question_data['answer_type'] == 'value':
            question_data['answer_unit'] = dict_question['answer']['answer_unit']
        if question_data['answer_type'] in ['binary','value']:
            question_data['answer_value'] = dict_question['answer']['answer_value']
            question_data['text'] = dict_question['question']
            question_data['context'] = dict_question['context']
            question_data['num_context'] = len(dict_question['context'])
            question_data['links'] = dict_question['question_links']
            question_data['num_context_links'] = len(dict_question['question_links'])
            num_main_passage = 0
            for context in dict_question['context']:
                if context['passage'] == 'main':
                    num_main_passage += 1
            question_data['num_context_main_passage'] = num_main_passage
            question_data['doc_num_links'] = len(json_data['links'])
            questions_data.append(question_data)

# Campos que começam com doc, se referenciam ao documento
# caso contrário às questões (respostas, contextos, etc)

In [67]:
# Criando o DataFrame df_questions
df_questions = pd.DataFrame(questions_data)

In [68]:
df_questions.shape

(343, 14)

In [69]:
# Exibindo o DataFrame resultante
df_questions.head()

,doc_pos,doc_title,doc_len_text,doc_text,answer_type,answer_unit,answer_value,text,context,num_context,links,num_context_links,num_context_main_passage,doc_num_links
0,1,Giovanni Messe,712,"Messe was born in Mesagne, in the Province of ...",value,years,5,How long had the First World War been over whe...,[{'text': 'he became aide-de-camp to King Vict...,2,[World War I],1,1,11
1,1,Giovanni Messe,712,"Messe was born in Mesagne, in the Province of ...",value,years,30,How old was Messe when the First World War sta...,"[{'text': 'Messe was born in Mesagne, in the P...",2,[World War I],1,1,11
2,2,Casino Royale (1967 film),1303,Casino Royale also takes credit for the greate...,value,years,2,How long had Angela Scoular been acting profes...,[{'text': 'Angela Scoular appeared as Ruby Bar...,3,"[Angela Scoular, On Her Majesty's Secret Servi...",2,1,22
3,4,Wilhelm Müller,961,Wilhelm Müller was born on 7 October 1794 at D...,value,years,9,How much time had passed between Wilhelm's ret...,[{'text': 'In 1814 he returned to his studies ...,2,[Max Müller],1,1,13
4,7,Turks and Caicos Islands national cricket team,1332,Cricket is the national sport of the Turks and...,value,year,1,How many years did the player who top-scored ...,[{'text': 'They were invited to take part in t...,3,[Donovan Matthews],1,2,14


Para simplificar pegaremos as com menor num_question_links + num_contex_links

In [163]:
df_questions['num_context'].describe()

count    343.0000000
mean       2.6151603
std        0.9259122
min        2.0000000
25%        2.0000000
50%        2.0000000
75%        3.0000000
max       11.0000000
Name: num_context, Length: 8, dtype: float64

In [70]:
round(df_questions['doc_len_text']/500).describe()

count    343.0000000
mean       2.4256560
std        0.9237829
min        1.0000000
25%        2.0000000
50%        2.0000000
75%        3.0000000
max        9.0000000
Name: doc_len_text, Length: 8, dtype: float64

In [71]:
df_questions['ind_complexidade'] = df_questions['num_context'] + df_questions['num_context_links'] + round(df_questions['doc_len_text']/500)

In [72]:
df_questions['ind_complexidade'].describe()

count    343.0000000
mean       6.3002915
std        1.5256769
min        4.0000000
25%        5.0000000
50%        6.0000000
75%        7.0000000
max       15.0000000
Name: ind_complexidade, Length: 8, dtype: float64

In [73]:
df_questions.sort_values(by='ind_complexidade', ascending=True)[df_questions['ind_complexidade']<=5].shape

/tmp/ipykernel_120029/3112322214.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_questions.sort_values(by='ind_complexidade', ascending=True)[df_questions['ind_complexidade']<=5].shape


(119, 15)

In [74]:
df_questions.sort_values(by='ind_complexidade', ascending=True)[(df_questions['ind_complexidade']<=5)].iloc[:20].groupby('answer_type').count()

/tmp/ipykernel_120029/1047843432.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_questions.sort_values(by='ind_complexidade', ascending=True)[(df_questions['ind_complexidade']<=5)].iloc[:20].groupby('answer_type').count()


,doc_pos,doc_title,doc_len_text,doc_text,answer_unit,answer_value,text,context,num_context,links,num_context_links,num_context_main_passage,doc_num_links,ind_complexidade
answer_type,,,,,,,,,,,,,,
binary,2,2,2,2,0,2,2,2,2,2,2,2,2,2
value,18,18,18,18,18,18,18,18,18,18,18,18,18,18


In [75]:
df_amostra_value = df_questions.sort_values(by='ind_complexidade', ascending=True)[(df_questions['ind_complexidade']<=5) & (df_questions['answer_type'] == 'value')].iloc[1:11]

/tmp/ipykernel_120029/1418523545.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_amostra_value = df_questions.sort_values(by='ind_complexidade', ascending=True)[(df_questions['ind_complexidade']<=5) & (df_questions['answer_type'] == 'value')].iloc[1:11]


In [76]:
df_amostra_binary = df_questions.sort_values(by='ind_complexidade', ascending=True)[(df_questions['ind_complexidade']<=5) & (df_questions['answer_type'] == 'binary')].iloc[:10]

/tmp/ipykernel_120029/222656906.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_amostra_binary = df_questions.sort_values(by='ind_complexidade', ascending=True)[(df_questions['ind_complexidade']<=5) & (df_questions['answer_type'] == 'binary')].iloc[:10]


In [77]:
df_amostra_binary.shape, df_amostra_value.shape

((10, 15), (10, 15))

In [78]:
df_amostra = pd.concat([df_amostra_binary, df_amostra_value])

In [ ]:
df_amostra['num_context_links'].describe()

count    20.0000000
mean      1.1000000
std       0.3077935
min       1.0000000
25%       1.0000000
50%       1.0000000
75%       1.0000000
max       2.0000000
Name: num_context_links, Length: 8, dtype: float64

In [138]:
df_amostra.to_csv(f"{DIRETORIO_TRABALHO}/df_amostra_value_binary.csv")

# Prompt Preparation

## Buscando um exemplo de value do próprio dataset

In [142]:
df_example_value = df_questions.sort_values(by='ind_complexidade', ascending=True)[(df_questions['ind_complexidade']<=5) & (df_questions['answer_type'] == 'value')].iloc[0].to_frame().T

/tmp/ipykernel_120029/2849726551.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_example_value = df_questions.sort_values(by='ind_complexidade', ascending=True)[(df_questions['ind_complexidade']<=5) & (df_questions['answer_type'] == 'value')].iloc[0].to_frame().T


In [143]:
df_example_value

,doc_pos,doc_title,doc_len_text,doc_text,answer_type,answer_unit,answer_value,text,context,num_context,links,num_context_links,num_context_main_passage,doc_num_links,ind_complexidade
0,1,Giovanni Messe,712,"Messe was born in Mesagne, in the Province of ...",value,years,5,How long had the First World War been over whe...,[{'text': 'he became aide-de-camp to King Vict...,2,[World War I],1,1,11,4.0


In [144]:
df_example_value['context'], df_example_value['text'], df_example_value['answer_value']

(0    [{'text': 'he became aide-de-camp to King Vict...
 Name: context, Length: 1, dtype: object,
 0    How long had the First World War been over whe...
 Name: text, Length: 1, dtype: object,
 0    5
 Name: answer_value, Length: 1, dtype: object)

In [171]:
documents_dict['World War I'][50:500]

'o known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918. Contemporaneously described as "the war to end all wars", it led to the mobilisation of more than 70 million military personnel, including 60 million Europeans, making it one of the largest wars in history. It is also one of the deadlie'

In [177]:
documents_dict['Giovanni Messe'][250:750]

't, he gave an important contribution to the creation and training of the "Arditi", elite infantry units, and with the rank of maggiore was the commander of the IX Nono Reparto Arditi that fought in the zone of Monte Grappa. Emerging considerably decorated from these conflicts, he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927. From this date until 1935, Messe commanded a unit of Bersaglieri and held the rank of colonel.\n'

## Construindo Prompt

In [167]:
STATIC_PROMPT = """For each example, use the documents to provide an answer to the question and cite evidence from the documents to support the answer. All questions have an answer, and just one answer.

Example 1:

Document 1: Title: Fred J. Shields. Content: Ollie Murphy\'s first-half goal gave \'the Royals\' a huge boost at half-time.

Document 2: Title: Ollie Murphy. Content: He plays club football for Carnaross

Document 3: Title: Ollie Murphy. Content: He came to national prominence in 1999 when he was one of Meath\'s best player\'s

Document 4: Title: The Thrilling World. Content: Sports encompass a wide range of physical activities that bring joy, competition, and excitement to millions of people worldwide. From team sports like football, basketball, and soccer to individual pursuits like tennis, golf, and swimming, sports offer a platform for athletes to showcase their skills and entertain spectators.

Document 5: Title: Family. Content: Family relationships encompass various dynamics, including the bond between parents and children, between siblings, and extended family members. 

Question: Based on the above documents, Did Ollie Murphy play for any teams other than \'the Royals\'?

Evidence: Document 1 says that Ollie Murphy\'s first-half goal gave \'the Royals\' a huge boost at half-time. However, this does not necessarily mean that Ollie only played for \'the Royals\'. Document 2 states that Ollie plays club football for Carnaross. This suggests that Ollie may have played for other teams in addition to \'the Royals\'. Document 3 says that Ollie came to national prominence in 1999 when he was one of Meath\'s best players. This also suggests that Ollie has played for other teams. Therefore, it is likely that Ollie has played for teams other than \'the Royals\'.

Answer: yes.

Example 2:

Document 1: Title: Terrible War. Content: The First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918. Contemporaneously described as "the war to end all wars", it led to the mobilisation of more than 70 million military personnel, including 60 million Europeans, making it one of the largest wars in history.

Document 2: Title: Ollie Murphy 2. Content: Ollie plays club football for Carnaross

Document 3: Title: The Thrilling World. Content: Sports encompass a wide range of physical activities that bring joy, competition, and excitement to millions of people worldwide. From team sports like football, basketball, and soccer to individual pursuits like tennis, golf, and swimming, sports offer a platform for athletes to showcase their skills and entertain spectators.

Document 4: Title: Miranda Cosgrove. Content: Messe gave an important contribution to the creation and training of the "Arditi", elite infantry units, and with the rank of maggiore was the commander of the IX Nono Reparto Arditi that fought in the zone of Monte Grappa. Emerging considerably decorated from these conflicts, he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927. From this date until 1935, Messe commanded a unit of Bersaglieri and held the rank of colonel.

Document 5: Title: Politics. Content: In the realm of politics, elected representatives, political parties, and government institutions strive to address societal issues, promote welfare, and uphold democratic principles. 

Question: How long had the First World War been over when Messe was named aide-de-camp?

Evidence: Document 1 indicates that Messe was named aide-de-camp to King Victor Emmanuel III between the years 1923 and 1927. Document 4 describes the First World War ended on 11 November 1918. Based on this information, we can calculate the time difference between the end of the First World War and Messe being named aide-de-camp. Since the war ended on 11 November 1918, and Messe was named aide-de-camp between 1923 and 1927, there is a time gap of 5 years between the end of the war and Messe assuming the role of aide-de-camp.

Answer: 5 years

Example 3:
"""



# Answer generation with ground truth documents

## Montando prompt ground truth

In [168]:
list_prompt_answer = []
for index, question in tqdm(df_amostra.iterrows()):
    min_total = 3
    prompt = STATIC_PROMPT
    for cnt, context in enumerate(question['context']):
        passage = context['passage'].replace('\n','')
        content = context['text'].replace('\n','')
        text = f"\nDocument {cnt+1}: Title: {passage}. Content: {content}\n"
        # print(text)
        prompt+= text
    
    prompt += f"\nQuestion: Based on the above documents, {question['text']}\n\nEvidence:"

    if question['answer_type'] == "value":
        resp = f"{question['answer_value']} {question['answer_unit']}"
    elif question['answer_type'] == "binary":
        resp = f"{question['answer_value']}"


    list_prompt_answer.append({'prompt': prompt, 'answer':resp})
    


20it [00:00, 5786.84it/s]


In [172]:
list_prompt_answer[0]

{'prompt': 'For each example, use the documents to provide an answer to the question and cite evidence from the documents to support the answer. All questions have an answer, and just one answer.\n\nExample 1:\n\nDocument 1: Title: Fred J. Shields. Content: Ollie Murphy\'s first-half goal gave \'the Royals\' a huge boost at half-time.\n\nDocument 2: Title: Ollie Murphy. Content: He plays club football for Carnaross\n\nDocument 3: Title: Ollie Murphy. Content: He came to national prominence in 1999 when he was one of Meath\'s best player\'s\n\nDocument 4: Title: The Thrilling World. Content: Sports encompass a wide range of physical activities that bring joy, competition, and excitement to millions of people worldwide. From team sports like football, basketball, and soccer to individual pursuits like tennis, golf, and swimming, sports offer a platform for athletes to showcase their skills and entertain spectators.\n\nDocument 5: Title: Family. Content: Family relationships encompass var

In [173]:
with open(f"{DIRETORIO_TRABALHO}/list_prompt_answer_ground_truth.pkl", 'wb') as outputFile:
    pickle.dump(list_prompt_answer, outputFile, pickle.HIGHEST_PROTOCOL)

# Answer generation with search for  documents

## Criando índice com passagens

In [117]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

In [118]:
%%time

def window(documents, stride=2, max_length=3):
    treated_documents = []

    for j,document in enumerate(tqdm(documents)):
        doc_text = document['content'].strip()
        doc = nlp(doc_text)
        sentences = [str(sent).strip() for sent in doc.sents]
        # print(f"len(sentences) {len(sentences)}")
        # print([(i,str(sent)) for i, sent in enumerate(sentences)])

        for i in range(0, len(sentences), stride):
            # print(f"i:i + max_length {i}:{i + max_length}")
            segment = ' '.join(sentences[i:i + max_length])
            treated_documents.append({
                "title": document['title'],
                "contents": document['title']+". "+segment,
                "segment": segment
            })
            if i + max_length >= len(sentences):
                break
    return treated_documents

treated_documents = window(documents, stride=2, max_length=3)

# na realidade o que repete é max_lengh-stride

100%|██████████| 7028/7028 [00:56<00:00, 124.95it/s]

CPU times: user 55.9 s, sys: 264 ms, total: 56.2 s
Wall time: 56.2 s


In [119]:
treated_documents[0], treated_documents[1] #, treated_documents[2]  treated_documents[3]

({'title': 'Palici',
  'contents': "Palici. The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia plain, and as a result these twin brothers were associated with geysers and the underworld.",
  'segment': "The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia plain, and as a result these twin brothers were associated with geysers and the underworld."},
 {'title': 'Palici',
  'contents': 'Palici. Their cult centered on three small lakes that emitt

In [ ]:
%%time
f = open(CAMINHO_CARDIN,'w')

for i, doc in enumerate(treated_documents):
    doc['id'] = i
    if doc['segment'] != "":
        f.write(json.dumps(doc)+"\n")

CPU times: user 2.15 s, sys: 257 ms, total: 2.41 s
Wall time: 2.41 s


In [ ]:
!python3 -m pyserini.index \
    -collection JsonCollection \
    -generator DefaultLuceneDocumentGenerator \
    -threads 8 \
    -input {DIRETORIO_CARDIN} \
    -index {DIRETORIO_INDICE}        
    # -storeRaw

pyserini.index is deprecated, please use pyserini.index.lucene.
2023-05-16 23:38:53,366 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-05-16 23:38:53,367 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-05-16 23:38:53,367 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-05-16 23:38:53,368 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /home/borela/fontes/deep_learning_em_buscas_unicamp/local/multidocqa/cardin
2023-05-16 23:38:53,368 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-05-16 23:38:53,368 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-05-16 23:38:53,368 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 8
2023-05-16 23:38:53,368 INFO  [main] index.

In [93]:
# Imprimindo estatísticas do índice
index_reader = IndexReader(DIRETORIO_INDICE)
print(index_reader.stats())

{'total_terms': 23832272, 'documents': 498078, 'non_empty_documents': 498078, 'unique_terms': 265045}


In [102]:
class MyDataset(Dataset):
    """
      Classe para representar um dataset de texto e classes.
    """  
    def __init__(self, texts: list, classes:list[int], tokenizer):
      """
      Inicializa um novo objeto MyDataset.

      Args:
          texts (list): uma lista com as strings de texto. Cada elemento deve ter 2 strings.
          classes (np.ndarray): um array com as classes de cada texto.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
          max_seq_length (int): o tamanho máximo da sequência a ser considerado.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      assert isinstance(texts, list), f"Parâmetro texts deve ser do tipo list e não {type(texts)}"
      for row in texts:
          assert isinstance(row, tuple) and len(row)== 2, f"Each row in texts must have 2 elements"
          assert isinstance(row[0], str) and isinstance(row[1], str), f"Each element in texts.row must be a string e não {type(row[0])}"
      assert isinstance(classes,np.ndarray), f'classes deve ser do tipo np.ndarray e não {type(classes)}'
      assert isinstance(classes[0],np.int64), f'classes[0] deve ser do tipo numpy.int64 e não {type(classes[0])} '

      self.texts = texts
      self.classes = classes
      self.tokenizer = tokenizer
      self.max_seq_length = tokenizer.model_max_length # model.config.max_position_embeddings
      if self.max_seq_length > 64000:
        print(f"Valor de self.max_seq_length  {self.max_seq_length} indica que deve ser usado outro campo do tokenizador. Assumido 512 ")
        self.max_seq_length =  512
      # Salvar os dados dos tensores
      x_data_input_ids = []
      x_data_token_type_ids = []
      x_data_attention_masks = []
      for text_pair in texts:
          encoding = tokenizer.encode_plus(
              text_pair[0],
              text_pair[1],
              add_special_tokens=True,
              max_length=self.max_seq_length,
              padding='max_length',
              return_tensors = 'pt',
              truncation=True,
              return_attention_mask=True,
              return_token_type_ids=True
          )
          x_data_input_ids.append(encoding['input_ids'].long())
          x_data_token_type_ids.append(encoding['token_type_ids'].long())
          x_data_attention_masks.append(encoding['attention_mask'].long())
      # print(F'\tVou converter lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      # squeeze: vai transformar um tensor de shape [2, 1, 322] em um tensor de shape [2, 322].

      self.x_tensor_input_ids = torch.stack(x_data_input_ids).squeeze(1)
      self.x_tensor_attention_masks = torch.stack(x_data_attention_masks).squeeze(1)
      self.x_tensor_token_type_ids = torch.stack(x_data_token_type_ids).squeeze(1)

    def __len__(self):
        """
          Retorna o tamanho do dataset (= tamanho do array texts)
        """
        return len(self.texts)
    
    def __getitem__(self, idx):
        """
          Retorna um dicionário com os dados do texto e sua classe correspondente, em um formato que pode 
          ser usado pelo dataloader do PyTorch para alimentar um modelo de aprendizado de máquina.
        """
        return {
            'input_ids': self.x_tensor_input_ids[idx],
            'attention_mask': self.x_tensor_attention_masks[idx],
            'token_type_ids': self.x_tensor_token_type_ids[idx],
            # 'labels': int(self.classes[idx])
            'labels': torch.tensor(self.classes[idx], dtype=torch.long)            
        }

In [151]:
def calcula_relevancia(parm_model, parm_dataloader_reranking, parm_model_name:str):
  # para 'cross-encoder/ms-marco-TinyBERT-L-2'
  prob_relevancia = []
  parm_model.eval()
  with torch.no_grad():
      for ndx, batch in enumerate(parm_dataloader_reranking):
          if 'minilm' in parm_model_name.lower():
              logits_model = parm_model(**BatchEncoding(batch).to(device)).logits                          
              relevantes_float = [float(t) for t in logits_model]
          elif 'monobert' in parm_model_name.lower():
              logits_model = parm_model(**BatchEncoding(batch).to(device)).logits
              probs = torch.nn.functional.softmax(logits_model, dim=1)
              nao_relevante, relevante = zip(*probs)
              relevantes_float = [float(t) for t in relevante]              
          prob_relevancia.extend(relevantes_float)          
          # prob_relevancia.append(pa.array(scores.numpy()))
  return prob_relevancia

In [174]:
def monta_prompt_com_busca_bm25_reranking(df_amostra, 
                                          searcher, 
                                          parm_model, 
                                          parm_tokenizer, 
                                          parm_model_name:str, 
                                          parm_documents_indexed:list,
                                          parm_num_max_doctos:int=15,
                                          num_max_hits=100):
    print(f'Running {df_amostra.shape[0]} queries in total')
    cnt = 0

    list_prompt_answer = []
    for index, question in tqdm(df_amostra.iterrows()):
        prompt = STATIC_PROMPT

       
        hits = searcher.search(question['text'], num_max_hits)
        # reranking
        docids = [hit.docid for hit in hits]
        pares_texto = [(question['text'], parm_documents_indexed[int(docid)]['contents']) for docid in docids]

        # print("carregando dataset")  
        classes_dummy = np.zeros(len(hits), dtype=np.int64)
        dataset_reranking = MyDataset(texts=pares_texto, classes=classes_dummy, tokenizer=parm_tokenizer)    
        dataloader_reranking = DataLoader(dataset_reranking,
                                            batch_size= 4,
                                            shuffle=False)
        prob_relevancia = calcula_relevancia(parm_model,dataloader_reranking, parm_model_name = parm_model_name)
        resultado_ordenado = sorted(zip(docids, prob_relevancia), key=lambda x: x[1], reverse=True)[:parm_num_max_doctos]
        assert len(resultado_ordenado) == parm_num_max_doctos, f"len(resultado_ordenado){len(resultado_ordenado)} deveria ser == parm_num_max_doctos {parm_num_max_doctos}"

        for cnt, (docid, prob) in enumerate(resultado_ordenado):
            passage = parm_documents_indexed[int(docid)]['title'].replace('\n','')
            content = parm_documents_indexed[int(docid)]['contents'].replace('\n','')
            text = f"\nDocument {cnt+1}: Title: {passage}. Content: {content}\n"
            # print(text)
            prompt+= text
        
        prompt += f"\nQuestion: Based on the above documents, {question['text']}\n\nEvidence:"

        if question['answer_type'] == "value":
            resp = f"{question['answer_value']} {question['answer_unit']}"
        elif question['answer_type'] == "binary":
            resp = f"{question['answer_value']}"


        list_prompt_answer.append({'prompt': prompt, 'answer':resp})

    return list_prompt_answer


### Experimentando bm25

In [158]:
searcher = LuceneSearcher(DIRETORIO_INDICE)

In [ ]:
test_question = 'How long had the First World War been over when Messe was named aide-de-camp?'

In [ ]:
hits = searcher.search(test_question, 100)
# reranking
docids = [hit.docid for hit in hits]

In [ ]:
docids[:5]

['1236', '477188', '477181', '477183', '1237']

In [156]:
treated_documents[1237], treated_documents[1236], treated_documents[1235]

({'title': 'Giovanni Messe',
  'contents': 'Giovanni Messe. Emerging considerably decorated from these conflicts, he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927. From this date until 1935, Messe commanded a unit of Bersaglieri and held the rank of colonel.',
  'segment': 'Emerging considerably decorated from these conflicts, he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927. From this date until 1935, Messe commanded a unit of Bersaglieri and held the rank of colonel.'},
 {'title': 'Giovanni Messe',
  'contents': 'Giovanni Messe. He saw action in the Italian conquest of Libya and in the First World War. During this conflict, he gave an important contribution to the creation and training of the "Arditi", elite infantry units, and with the rank of maggiore was the commander of the IX Nono Reparto Arditi that fought in the zone of Monte Grappa. Emerging considerably decorated from these conflicts, he became aid

## bm25@100_reranking_minilm

In [108]:
# nome_modelo = 'cross-encoder/ms-marco-TinyBERT-L-2'
nome_modelo = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
model = AutoModelForSequenceClassification.from_pretrained(nome_modelo).to(device)
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

In [109]:
model.config.max_position_embeddings, tokenizer.max_len_sentences_pair, tokenizer.model_max_length

(512, 509, 512)

In [145]:
df_example_value

,doc_pos,doc_title,doc_len_text,doc_text,answer_type,answer_unit,answer_value,text,context,num_context,links,num_context_links,num_context_main_passage,doc_num_links,ind_complexidade
0,1,Giovanni Messe,712,"Messe was born in Mesagne, in the Province of ...",value,years,5,How long had the First World War been over whe...,[{'text': 'he became aide-de-camp to King Vict...,2,[World War I],1,1,11,4.0


Teste com amostra exemplo

In [159]:
list_prompt_answer_search = monta_prompt_com_busca_bm25_reranking(df_example_value, 
                                          searcher, 
                                          model, 
                                          tokenizer, 
                                          nome_modelo,  
                                          parm_documents_indexed=treated_documents,
                                          parm_num_max_doctos=15,
                                          num_max_hits=100)

Running 1 queries in total


1it [00:00,  1.69it/s]


Rodando para amostra (20 exemplos)

In [175]:
list_prompt_answer_search = monta_prompt_com_busca_bm25_reranking(df_amostra, 
                                          searcher, 
                                          model, 
                                          tokenizer, 
                                          nome_modelo,  
                                          parm_documents_indexed=treated_documents,
                                          parm_num_max_doctos=5,
                                          num_max_hits=100)

Running 20 queries in total


20it [00:10,  1.91it/s]


In [176]:
with open(f"{DIRETORIO_TRABALHO}/list_prompt_answer_search.pkl", 'wb') as outputFile:
    pickle.dump(list_prompt_answer_search, outputFile, pickle.HIGHEST_PROTOCOL)